In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.metrics import log_loss
import ast

# gets rid of scientific notation of arrays when printing
np.set_printoptions(suppress=True)

# shows all columns 
pd.set_option('display.max_columns', None) 

In [ ]:
learning_rate = .00001

In [ ]:
## Define activation functions
# softmax activation function
def softmax(x):
    expx = np.exp(x - np.max(x))
    return expx/np.sum(expx, axis=1, keepdims=True)

# relu activation function
def relu(x):
    return np.maximum(0, x)

# derivative of relu activation function
def drelu(x):
    return np.where(x > 0, 1, 0)

In [ ]:
def classification_accuracy(targets, predictions) :
    total = len(predictions)
    correct = 0
    for i in range(total):
        predicted_class = np.argmax(predictions[i])
        if targets[i][predicted_class] == 1:
           correct += 1
    return correct/total

In [ ]:
donordata = pd.read_csv('./data/clean-data.csv', index_col=None)

traindata = donordata.sample(frac=.8, random_state=1)
testdata = donordata.drop(traindata.index)

train_target_output = traindata.target_output.values
test_target_output = testdata.target_output.values

traindata = traindata.drop(['target_output'], axis=1)
traindata = traindata.set_index(['Contact_ID'], drop=True)
testdata = testdata.drop(['target_output'], axis=1)
testdata = testdata.set_index(['Contact_ID'], drop=True)

In [ ]:
# one hot encode the target output for the train data
first = True
for x in train_target_output:
    if first:
        train_one_hot_encoded = np.array([ast.literal_eval(x)])
        first = False
    else:
        train_one_hot_encoded = np.append(train_one_hot_encoded, [ast.literal_eval(x)],axis=0)

In [ ]:
# one hot encode the target output for the test data
first = True
for x in test_target_output:
    if first:
        test_one_hot_encoded = np.array([ast.literal_eval(x)])
        first = False
    else:
        test_one_hot_encoded = np.append(test_one_hot_encoded, [ast.literal_eval(x)],axis=0)

In [ ]:
num_inputs = len(traindata.columns)

# initialize weights using He initialization
np.random.seed(1)
weights_1 = np.random.randn(num_inputs,40)*np.sqrt(2/num_inputs)
weights_2 = np.random.randn(40,40)*np.sqrt(2/40)
weights_3 = np.random.randn(40,40)*np.sqrt(2/40)
weights_4 = np.random.randn(40,14)*np.sqrt(2/40)

# initialize bias
bias_1 = 0
bias_2 = 0
bias_3 = 0
bias_4 = 0

In [ ]:
## takes in the inputs and adjusts the weights accordingly
# If train is false, it makes a prediction and doesn't adjust the weights
def train(batch, target_outputs, train):
    global weights_4
    global weights_3
    global weights_2
    global weights_1
    global bias_1
    global bias_2
    global bias_3
    global bias_4
    
    ## feedforward
    layer_1_output = relu(np.dot(batch, weights_1) + bias_1)
    layer_2_output = relu(np.dot(layer_1_output, weights_2) + bias_2)
    layer_3_output = relu(np.dot(layer_2_output, weights_3) + bias_3)
    output = softmax(np.dot(layer_3_output, weights_4) + bias_4)

    if train:
        ## backprop
        # output layer
        dcost_dpred = target_outputs - output
        dz_dw4 = layer_3_output
        
        dcost_dw4 = np.dot(dz_dw4.T, dcost_dpred)
        dcost_b4 = dcost_dpred
        
        # layer 3
        dcost_doutput3 = np.dot(dcost_dpred, weights_4.T)
        doutput3_dz = drelu(layer_3_output)
        dz_dw3 = layer_2_output
        
        dcost_dw3 = np.dot(dz_dw3.T, dcost_doutput3 * doutput3_dz)
        dcost_b3 = dcost_doutput3 * doutput3_dz
        
        # layer 2
        dcost_doutput2 = np.dot(dcost_doutput3 * doutput3_dz, weights_3.T)
        doutput2_dz = drelu(layer_2_output)
        dz_dw2 = layer_1_output
        
        dcost_dw2 = np.dot(dz_dw2.T, dcost_doutput2 * doutput2_dz)
        dcost_b2 = dcost_doutput2 * doutput2_dz

        # layer 1
        dcost_doutput1 = np.dot(dcost_doutput2 * doutput2_dz, weights_2.T)
        doutput1_dz = drelu(layer_1_output)
        dz_dw1 = batch
        
        dcost_dw1 = np.dot(dz_dw1.T, dcost_doutput1 * doutput1_dz)
        dcost_b1 = dcost_doutput1 * doutput1_dz
        
        # update weights
        weights_4 += dcost_dw4 * learning_rate
        weights_3 += dcost_dw3 * learning_rate
        weights_2 += dcost_dw2 * learning_rate
        weights_1 += dcost_dw1 * learning_rate

        # update biases
        bias_4 += np.average(dcost_b3) * learning_rate
        bias_3 += np.average(dcost_b3) * learning_rate
        bias_2 += np.average(dcost_b2) * learning_rate
        bias_1 += np.average(dcost_b1) * learning_rate
        
    return output

In [ ]:
error_list = []
index = []
train_data = preprocessing.scale(traindata.values)
test_data = preprocessing.scale(testdata.values)
for i in range(1001):
    train_output = train(train_data, train_one_hot_encoded, True)
    error = log_loss(train_one_hot_encoded, train_output)
    error_list.append(error)
    index.append(i)
    if (i % 100) == 0:
        test_output = train(test_data, test_one_hot_encoded, False)
        accuracy = classification_accuracy(test_one_hot_encoded, test_output)
        print(accuracy)

In [ ]:
# plot error vs. epoch
plt.plot(index, error_list)
plt.xlabel('Epoch')
plt.ylabel('Cross Entropy Loss')
plt.title('Error')
plt.show()

In [ ]:
# classification accuracy of training data
classification_accuracy(train_one_hot_encoded, train_output)

In [ ]:
# classification accuracy of test data
test_data = preprocessing.scale(testdata.values)
test_output = train(test_data, test_one_hot_encoded, False)
classification_accuracy(test_one_hot_encoded, test_output)